# <center>Projeto Big Data</center> 
&nbsp;
#### **<center>Otimização da frota de ônibus turísticos</center>**
<br />
<br />
<br />
<br />
<center>Daniel Castro</center>
&nbsp;
<center>Theo Barbara</center>
<br />
<br />
<center>São Paulo | Nov 2022</center>
<br />

### Introdução 
<br />
<br />
    Este projeto tem como intuito a análise das rotas mais procuradas ao longo do ano a fim de possibilitar a alocação mais eficaz e lucrativa das frotas de ônibus. Ao ter conhecimento das rotas mais buscadas pelos passageiros é possível transferir ônibus que levariam bem menos passageiros para estas rotas com maior demanda, possibilitando uma maior captura de passageiros e consequentemente um lucro maior. Foram utilizadas para este estudo as bases de dados da Agência Nacional de Transportes Terrestres (ANTT), o link para o acesso às bases segue abaixo: 
<br />
<br />

[Base de dados ANTT](https://dados.antt.gov.br/dataset/monitriip-bilhetes-de-passagem)

<br />
<br />

### Organização dos dados
<br />
<br />
    Ao entrar no site da ANTT você vai perceber que os dados estão separados por mês, portanto para ter um volume de dados adequado para a análise foi preciso baixar alguns meses. Estes arquivos CSV de cada mês foram alocados em uma pasta e compactados para diminuir ligeiramente o tamanho do arquivo de trabalho.
<br />
<br />
    Para trabalhar com este arquivo ZIP com todos os meses a biblioteca ZipFile foi utilizada. O procedimento de análise dos dados foi basicamente abrir o arquivo ZIP, percorrer todos os arquivos que o compõem e logo após uní-los em um arquivo só.
<br />
<br />

In [18]:
from zipfile import ZipFile
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
  
# Definindo o nome do arquivo de análise e seu caminho.
file_name = "Bilhetes.zip"
#path = "C:/Users/danie/Desktop/Estudo/Eletivas/Big Data para dados públicos/Projeto-Big-Data/"
    
# Abrindo a arquivo zip com as informações de todos os meses
with ZipFile(os.path.join(file_name), 'r') as zip:
    # Printando o nome de todos os arquivo dentro do zip
    zip.printdir()  
    # Extraindo os arquivos para o diretório de trabalho
    zip.extractall()

File Name                                             Modified             Size
venda_passagem_12_2021.csv                     2022-11-09 12:37:42     19642786
venda_passagem_01_2019.csv                     2022-11-09 14:21:10     10164393
venda_passagem_01_2020.csv                     2022-11-09 14:20:06     16489544
venda_passagem_01_2021.csv                     2022-11-09 14:18:56     14744417
venda_passagem_01_2022.csv                     2022-11-09 12:37:40     17167260
venda_passagem_02_2019.csv                     2022-11-09 14:21:04      8703721
venda_passagem_02_2020.csv                     2022-11-09 14:20:02     15885862
venda_passagem_02_2021.csv                     2022-11-09 14:18:50     13524920
venda_passagem_02_2022.csv                     2022-11-09 12:37:40     16620848
venda_passagem_03_2019.csv                     2022-11-09 14:21:00      8282801
venda_passagem_03_2020.csv                     2022-11-09 14:19:52     11977390
venda_passagem_03_2021.csv              

In [19]:
# Criando o Data Frame de análise em branco e depois fazendo a inserção das informações de todos
# os meses
print('Construindo o arquivo...')
bilhetes = pd.DataFrame()
for file in zip.namelist():
    bilheteMes = pd.read_csv(file, sep=";", encoding="latin-1")
    try:
        bilhetes = pd.concat([bilhetes, bilheteMes], ignore_index=True)
        # Mostrando a evolução do tamanho da tabela 
        print(f"'{len(bilhetes)}' linhas.") 
    except Exception as error:
        print(error)
print(f"Finalizado, o arquivo possui '{len(bilhetes)}' linhas!'")

Construindo o arquivo...
'152614' linhas.
'229434' linhas.
'353000' linhas.
'465993' linhas.
'598150' linhas.
'664301' linhas.
'783998' linhas.
'887449' linhas.
'1015623' linhas.
'1077900' linhas.
'1167992' linhas.
'1269001' linhas.
'1399756' linhas.
'1479005' linhas.
'1499676' linhas.
'1601342' linhas.
'1668240' linhas.
'1760529' linhas.
'1781252' linhas.
'1888904' linhas.
'2017874' linhas.
'2105757' linhas.
'2137514' linhas.
'2246516' linhas.
'2373333' linhas.
'2473717' linhas.
'2519192' linhas.
'2639491' linhas.
'2792982' linhas.
'2878156' linhas.
'2944846' linhas.
'3067216' linhas.
'3164334' linhas.
'3237633' linhas.
'3359004' linhas.
'3494567' linhas.
'3597029' linhas.
'3692555' linhas.
'3816802' linhas.
'3915555' linhas.
'4018278' linhas.
'4144253' linhas.
'4273578' linhas.
'4391804' linhas.
Finalizado, o arquivo possui '4391804' linhas!'


In [20]:
# Limpando linhas com valores NaN
if len(bilhetes[bilhetes.isna().any(axis=1)]) != 0:
    print(f"O arquivo contém '{len(bilhetes[bilhetes.isna().any(axis=1)])}' linhas com NaN que"+
          "devem ser apagadas")
    print("Removendo linhas NaN...")
    bilhetes.dropna(inplace=True)
print(f"O arquivo contém '{len(bilhetes[bilhetes.isna().any(axis=1)])}' linhas com NaN!")

O arquivo contém '0' linhas com NaN!


In [21]:
bilhetes.sample(5)

,mes_emissao_bilhete,mes_viagem,ponto_origem_viagem,ponto_destino_viagem,tipo_servico,tipo_gratuidade,media_valor_total,dp_valor_total,quantidade_bilhetes
3521594,10/2019,10/2019,ITAOBIM/MG,MARINGA/PR,Convencional com sanitário,"Bilhete de Viagem do Idoso 50% - Inciso II, ar...",217.20,0.00,1
2407432,07/2019,07/2019,IRAI/RS,BARRACAO/PR,Convencional com sanitário,Tarifa Normal - sem desconto,34.04,0.00,2
3957791,11/2020,11/2020,PICOS/PI,TAUA/CE,Convencional com sanitário,"Tarifa Promocional - Parágrafo 3º, art. 27 do ...",29.17,29.31,14
254915,01/2020,01/2020,VENANCIO AIRES/RS,ARARANGUA/SC,Convencional com sanitário,Tarifa Normal - sem desconto,80.48,2.72,17
1204997,03/2021,03/2021,LAGES/SC,SANTO ANGELO/RS,Convencional com sanitário,Gratuidade Jovem de Baixa Renda 50% - Inciso I...,59.65,0.00,3


Perceba que nas colunas "_mes_emissao_bilhete_" e "_mes_viagem_" a informação da data pode ser apresentada em dois formatos diferentes: totalmente numérico, ou seja, **09/2022**, ou então utilizando a sigla do mês e uma abreviação do ano, ou seja, **set/22**. 

<br />
É preciso unificar a apresentação das informações para que a análise não seja atrapalhada por esta divergência de formato. O formato escolhido foi o númerico, logo abaixo será feita a conversão das informações das colunas.

<br />
<br />

**É imprescindível que as informações de data contenham apenas mês e ano.**

In [22]:
def conversorData(data:str):
    """
    Converte as datas que estão em um foramto diferente do esperado.
    :param data: data a ser convertida
    return: 
    - Caso data seja passível de conversão, retorna a data convertida no formato MM/AAAA.
    
    - Caso contrário, retorna "Informação inválida" 
    """        
    
    mes, ano = data.split('/')
    try:
        int(mes)
    except Exception as error:
        if type(error) is ValueError: # Significa que existe alguma letra
            mes = mes.lower()
            # Faz a respetiva atribuição numérica da sigla do mês da data
            if mes == 'jan':
                mes = '01'
            elif mes == 'fev':
                mes = '02'
            elif mes == 'mar':
                mes = '03'
            elif mes == 'abr':
                mes = '04'
            elif mes == 'mai':
                mes = '05'
            elif mes == 'jun':
                mes = '06'
            elif mes == 'jul':
                mes = '07'
            elif mes == 'ago':
                mes = '08'
            elif mes == 'set':
                mes = '09'
            elif mes == 'out':
                mes = '10'
            elif mes == 'nov':
                mes = '11'
            elif mes == 'dez':
                mes = '12'
            else:
                return 'Informação inválida' 
    else:
        if int(mes) not in range(1,13): # o mês não é válida
            return 'Informação inválida' 
    finally:
        if len(ano) == 2: # A data está no formato MM/AA e deve ser convertida para MM/AAAA
            if int(ano) in range(0,25):
                ano = '20'+ano
            else:
                ano = '19'+ano
        return mes+'/'+ano

In [23]:
# Conversão das colunas de data
# A linha abaixo possui o formato errado de data.
print('Convertendo...')
bilhetes['mes_viagem'] = bilhetes['mes_viagem'].apply(lambda x: conversorData(x))
bilhetes['mes_emissao_bilhete'] = bilhetes['mes_emissao_bilhete'].apply(lambda x: conversorData(x))
print("As colunas 'mes_viagem' e 'mes_emissao_bilhete' foram convertidas!")

Convertendo...
As colunas 'mes_viagem' e 'mes_emissao_bilhete' foram convertidas!


### Análise exploratória
<br />
Para começar é interessante observar graficamente para um mês qualquer, qual a rota que possui a maior quantidade de bilhetes e consequentemente a maior quantidade de passageiros. 

Vou utilizar o mês do meu aniversário só por coincidência.

In [24]:
def analiseExploratoria(df:pd.DataFrame, data:str,  numAmostras:int, sentido:bool=False):
    """
    A função calcula o número de bilhetes para os parâmetros de entrada.
    :param df: Data Frame com as informações dos bilhetes
    :param data: data de análise, deve estar no formato MM/AAAA.
    :param sentido: True | Origem
                    False | Destino
    :param numAmostras: Quantidade de cidades que serão apresentadas no gráfico
    """
    
    if sentido:
        sentido = 'ponto_origem_viagem'
        titulo = 'Cidades de origem com maior número de passageiros em '+data
    else:
        sentido = 'ponto_destino_viagem'
        titulo = 'Cidades de destino com maior número de passageiros em '+data
        
    Bilhetes = df[(df['mes_viagem'].str[0:8] == data)]
    Bilhetes = Bilhetes.groupby([sentido]).count()
    Bilhetes = Bilhetes.sort_values(by=['mes_viagem'], ascending=False).head(numAmostras)
    
    fig = px.bar(Bilhetes, y='mes_viagem',
                  labels={'mes_viagem': 'N° bilhetes emitidos', 
                          'ponto_origem_viagem':'Cidade de origem',
                          'ponto_destino_viagem': 'Cidade de destino'},
                  title=titulo,
                  template='plotly_white')   
    
    return fig.show()                                        

In [82]:
bilhetes.groupby(['ponto_origem_viagem']).count()

,mes_emissao_bilhete,mes_viagem,ponto_destino_viagem,tipo_servico,tipo_gratuidade,media_valor_total,dp_valor_total,quantidade_bilhetes
ponto_origem_viagem,,,,,,,,
ABADIA DOS DOURADOS/MG,3,3,3,3,3,3,3,3
ABADIANIA/GO,804,804,804,804,804,804,804,804
ABAIRA/BA,2,2,2,2,2,2,2,2
ABELARDO LUZ/SC,2155,2155,2155,2155,2155,2155,2155,2155
ABREU E LIMA/PE,663,663,663,663,663,663,663,663
...,...,...,...,...,...,...,...,...
XINGUARA/PA,2013,2013,2013,2013,2013,2013,2013,2013
XIQUE-XIQUE/BA,1128,1128,1128,1128,1128,1128,1128,1128
ZACARIAS/SP,11,11,11,11,11,11,11,11


In [25]:
analiseExploratoria(df=bilhetes, data='04/2022', numAmostras=10, sentido=True)

In [26]:
analiseExploratoria(df=bilhetes, data='04/2022', numAmostras=10, sentido=False)

Perceba que tanto no ponto de partida, quanto no ponto de destino as cinco cidades predominantes são:

<br />

 *  São Paulo (SP)
 *  Rio de Janeiro (RJ)
 *  Goiânia (GO)
 *  Brasília (DF)
 *  Curitiba (PR)
 
<br />

Com certeza estas são cidades boas para se inserir em suas rotas de viagem. 

Mas existe também duas outras perguntas: 

<br />

 *  "Estas pessoas que saem de São Paulo, para onde elas vão?"

<br />

 *  "E as pessoas que vêm para São Paulo, de onde elas partem?"
 
<br />

Para responder a estas perguntas vamos fazer uma cross table e ver o resultado.

In [27]:
def rotas(df:pd.DataFrame, data:str,  numAmostras:int, cidade:str, sentido:bool=False):
    """
    A função calcula o número de bilhetes para os parâmetros de entrada.
    :param df: Data Frame com as informações dos bilhetes.
    :param data: data de análise, deve estar no formato MM/AAAA.
    :param sentido: True | Cidade é origem.
                    False | Cidade é destino.
    :param numAmostras: Quantidade de cidades que serão apresentadas no gráfico.
    :param cidade: Cidade da análise, deve estar no formato nomeCidade/UF.
    :return: Um gráfico.
    """
        
    if sentido:
        ponto1 = 'ponto_origem_viagem'
        ponto2 = 'ponto_destino_viagem'
        titulo = 'Rotas com sentido a '+ cidade + ' em ' + data
    else:
        ponto2 = 'ponto_origem_viagem'
        ponto1 = 'ponto_destino_viagem'
        titulo = 'Rotas saindo de '+ cidade + ' em ' + data
        
    Bilhetes = df[(df['mes_viagem'].str[0:8] == data)]
    rotas = pd.crosstab(Bilhetes[ponto1],Bilhetes[ponto2])
    rotas =rotas.loc[:,[cidade]].sort_values(by=[cidade], ascending = False).head(numAmostras)
    rotas = rotas.loc[rotas[cidade] != 0] # Retirando as cidades que não possuem bilhetes
    
    
    fig = px.bar(rotas, title=titulo,
                 labels={'value':'N° de bilhetes', 'ponto_origem_viagem': 'Cidade de origem', 
                        'ponto_destino_viagem': 'Cidade de destino'},
                 template='plotly_white')
    return fig.show()           

### As 10 cidades que mais possuem passageiros em direção a São Paulo em abril são:

In [28]:
rotas(df=bilhetes, data='04/2022', numAmostras=10, sentido=True, cidade='SAO PAULO/SP')

### As 10 cidades que os passageiros de São Paulo mais viajam em abril são:

In [29]:
rotas(df=bilhetes, data='04/2022', numAmostras=10, sentido=False, cidade='SAO PAULO/SP')

### Função de análise temporal

A partir da análise feita acima foi possível identificar os melhores trajetos para o mês de abril de 2022, mas será que são sempre os mesmos trajetos que se destacam todos os anos? Na base de dados da ANTT têm-se dados de Janeiro de 2019 até Setembro de 2022. 

<br /> 
Abaixo será feita a análise da quantidade de bilhetes no mês de Abril dos anos 2019 até 2022.

In [52]:
def analiseTemporal(df:pd.DataFrame, mes:int,  numAmostras:int, sentido:bool=False, 
                    anos:list=['2019', '2020', '2021', '2022'], stacked:bool=False):
    """
    A função calcula o número de bilhetes ao redor dos anos para os parâmetros de entrada.
    :param df: Data Frame com as informações dos bilhetes
    :param mes: Mês de análise.
    :param sentido: True | Origem
                    False | Destino
    :param numAmostras: Quantidade de cidades que serão apresentadas no gráfico
    :param anos: Lista dos anos que se deseja observar.
    """
    mes = '0'+str(mes)
    if sentido:
        sentido = 'ponto_origem_viagem'
        titulo = 'Cidades de origem com maior número de passageiros em '+mes
    else:
        sentido = 'ponto_destino_viagem'
        titulo = 'Cidades de destino com maior número de passageiros em '+mes
    
    Bilhetes = {}
    
    for ano in anos:
        Bilhetes[ano] = df[(df['mes_viagem'].str[0:7] == mes+'/'+ano)]

    for ano in Bilhetes:
        Bilhetes[ano] = Bilhetes[ano].groupby([sentido]).count()
        Bilhetes[ano].rename(columns = {'mes_viagem':ano}, inplace = True)
        Bilhetes[ano] = Bilhetes[ano][ano]

    Bilhetes = pd.DataFrame(Bilhetes)
    Bilhetes['Total']=Bilhetes.apply(np.sum, axis=1)
    Bilhetes = Bilhetes.sort_values(by=['Total'], ascending=False).head(numAmostras)
    if stacked:
        fig = px.bar(Bilhetes, y=anos, 
                     title=titulo,
                     labels={'value': 'N° de bilhetes', 'ponto_origem_viagem': 'Cidade de origem',
                            'ponto_destino_viagem': 'Cidade de destino', 'variable': 'Ano'})
    else:
        Bilhetes = Bilhetes.T
        fig = go.Figure()
        for ano in Bilhetes:
            fig.add_trace(go.Bar(
                x=anos,
                y=Bilhetes[ano],
                name=ano))
        fig.update_layout(barmode='group', xaxis_tickangle=-45)
    return fig.show()

In [53]:
analiseTemporal(df=bilhetes, mes=4, numAmostras=5, stacked=False)

In [54]:
analiseTemporal(df=bilhetes, mes=4, numAmostras=5)

ANÁLISE PREDITIVA

SEPARAÇÃO DE TREINO E TESTE

In [ ]:
df_train, df_teste = 

In [55]:
bilhetes.head(5)

,mes_emissao_bilhete,mes_viagem,ponto_origem_viagem,ponto_destino_viagem,tipo_servico,tipo_gratuidade,media_valor_total,dp_valor_total,quantidade_bilhetes
0,12/2021,12/2021,PARNAIBA/PI,FORTALEZA/CE,Leito com ar condicionado,"Tarifa Promocional - Parágrafo 3º, art. 27 do ...",120.34,16.77,438
1,12/2021,12/2021,PALMITOS/SC,SARANDI/RS,Convencional com sanitário,Tarifa Normal - sem desconto,39.00,0.00,1
2,12/2021,12/2021,MACAE/RJ,BELO HORIZONTE/MG,Convencional com sanitário,Tarifa Normal - sem desconto,0.00,0.00,1
3,12/2021,12/2021,BRASILIA/DF,VIANOPOLIS/GO,Convencional com sanitário,"Tarifa Promocional - Parágrafo 3º, art. 27 do ...",50.44,31.34,8
4,12/2021,12/2021,BRASILIA/DF,JUIZ DE FORA/MG,Convencional com sanitário,"Tarifa Promocional - Parágrafo 3º, art. 27 do ...",107.00,109.14,8


In [64]:
df_train = bilhetes[(bilhetes['mes_viagem'].str[3:7] != "2022")]

df_test = bilhetes[(bilhetes['mes_viagem'].str[3:7] == "2022")]

In [65]:
df_train.shape

(3340571, 9)

In [66]:
df_test.shape

(1051233, 9)

In [67]:
bilhetes.shape

(4391804, 9)

In [69]:
!pip install pyspark

  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845525 sha256=09418ec6c6a40c53bff1a67bd9aa36a339acd63f1ba72060a0938f1c2a52fd0d
  Stored in directory: c:\users\theob\appdata\local\pip\cache\wheels\51\c8\18\298a4ced8ebb3ab8a7d26a7198c0cc7035abb906bde94a4c4b
Successfully built pyspark


In [73]:
from pyspark.ml.regression import LinearRegression
from sklearn import linear_model

#linear_reg = linear_model.LinearRegression()

ValueError: could not convert string to float: '12/2021'